## Football Attendace Data Wrangling
    By: Jack Guptill
    Date: 11/12/23

Importing Needed Libraries

In [3]:
import pandas as pd
import matplotlib.pyplot as mp
import seaborn as sns

Importing Raw Dataframes

In [11]:
attendance = pd.read_csv('attendance.csv')
games = pd.read_csv('games.csv')
standings = pd.read_csv('standings.csv')

In [13]:
attendance.head()

,team,team_name,year,total,home,away,week,weekly_attendance
0,Arizona,Cardinals,2000,893926,387475,506451,1,77434.0
1,Arizona,Cardinals,2000,893926,387475,506451,2,66009.0
2,Arizona,Cardinals,2000,893926,387475,506451,3,NaN
3,Arizona,Cardinals,2000,893926,387475,506451,4,71801.0
4,Arizona,Cardinals,2000,893926,387475,506451,5,66985.0


In [14]:
games.head()

,year,week,home_team,away_team,winner,tie,day,date,time,pts_win,pts_loss,yds_win,turnovers_win,yds_loss,turnovers_loss,home_team_name,home_team_city,away_team_name,away_team_city
0,2000,1,Minnesota Vikings,Chicago Bears,Minnesota Vikings,NaN,Sun,September 3,1:00PM,30,27,374,1,425,1,Vikings,Minnesota,Bears,Chicago
1,2000,1,Kansas City Chiefs,Indianapolis Colts,Indianapolis Colts,NaN,Sun,September 3,1:00PM,27,14,386,2,280,1,Chiefs,Kansas City,Colts,Indianapolis
2,2000,1,Washington Redskins,Carolina Panthers,Washington Redskins,NaN,Sun,September 3,1:01PM,20,17,396,0,236,1,Redskins,Washington,Panthers,Carolina
3,2000,1,Atlanta Falcons,San Francisco 49ers,Atlanta Falcons,NaN,Sun,September 3,1:02PM,36,28,359,1,339,1,Falcons,Atlanta,49ers,San Francisco
4,2000,1,Pittsburgh Steelers,Baltimore Ravens,Baltimore Ravens,NaN,Sun,September 3,1:02PM,16,0,336,0,223,1,Steelers,Pittsburgh,Ravens,Baltimore


In [15]:
standings.head()

,team,team_name,year,wins,loss,points_for,points_against,points_differential,margin_of_victory,strength_of_schedule,simple_rating,offensive_ranking,defensive_ranking,playoffs,sb_winner
0,Miami,Dolphins,2000,11,5,323,226,97,6.1,1.0,7.1,0.0,7.1,Playoffs,No Superbowl
1,Indianapolis,Colts,2000,10,6,429,326,103,6.4,1.5,7.9,7.1,0.8,Playoffs,No Superbowl
2,New York,Jets,2000,9,7,321,321,0,0.0,3.5,3.5,1.4,2.2,No Playoffs,No Superbowl
3,Buffalo,Bills,2000,8,8,315,350,-35,-2.2,2.2,0.0,0.5,-0.5,No Playoffs,No Superbowl
4,New England,Patriots,2000,5,11,276,338,-62,-3.9,1.4,-2.5,-2.7,0.2,No Playoffs,No Superbowl


EDA

In [24]:
attendance.info()
attendance.isna().sum()

# Weekly attendance is a little strange but this might be due to bye weeks. 
# Will need to investigate this...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10846 entries, 0 to 10845
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   team               10846 non-null  object 
 1   team_name          10846 non-null  object 
 2   year               10846 non-null  int64  
 3   total              10846 non-null  int64  
 4   home               10846 non-null  int64  
 5   away               10846 non-null  int64  
 6   week               10846 non-null  int64  
 7   weekly_attendance  10208 non-null  float64
dtypes: float64(1), int64(5), object(2)
memory usage: 678.0+ KB


team                   0
team_name              0
year                   0
total                  0
home                   0
away                   0
week                   0
weekly_attendance    638
dtype: int64

In [31]:
attendance[attendance['weekly_attendance'].isna()]

# There doesnt seem to be duplicated team in the same year so I would assume they are NA due to bye weeks. 

,team,team_name,year,total,home,away,week,weekly_attendance
2,Arizona,Cardinals,2000,893926,387475,506451,3,NaN
31,Atlanta,Falcons,2000,964579,422814,541765,15,NaN
47,Baltimore,Ravens,2000,1062373,551695,510678,14,NaN
54,Buffalo,Bills,2000,1098587,560695,537892,4,NaN
71,Carolina,Panthers,2000,1095192,583489,511703,4,NaN
...,...,...,...,...,...,...,...,...
10764,San Francisco,49ers,2019,1080454,562443,518011,4,NaN
10788,Seattle,Seahawks,2019,1101649,551927,549722,11,NaN
10801,Tampa Bay,Buccaneers,2019,949068,415189,533879,7,NaN
10822,Tennessee,Titans,2019,1047496,516074,531422,11,NaN


In [38]:
games.info()
games.isna().sum()

games.shape

# Will need to verify that all NA's from ties are due to a winner actually being declared

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5324 entries, 0 to 5323
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year            5324 non-null   int64 
 1   week            5324 non-null   object
 2   home_team       5324 non-null   object
 3   away_team       5324 non-null   object
 4   winner          5324 non-null   object
 5   tie             10 non-null     object
 6   day             5324 non-null   object
 7   date            5324 non-null   object
 8   time            5324 non-null   object
 9   pts_win         5324 non-null   int64 
 10  pts_loss        5324 non-null   int64 
 11  yds_win         5324 non-null   int64 
 12  turnovers_win   5324 non-null   int64 
 13  yds_loss        5324 non-null   int64 
 14  turnovers_loss  5324 non-null   int64 
 15  home_team_name  5324 non-null   object
 16  home_team_city  5324 non-null   object
 17  away_team_name  5324 non-null   object
 18  away_tea

(5324, 19)

In [62]:
games[games['tie'].notna()] #verified these games to be true ties

# Going to replace all NA's with 0's and will make all ties with a 1. 

games.loc[games['tie'].notna(), 'tie'] = 1
games.loc[games['tie'].isna(), 'tie'] = 0

# Verifying with an example game Green Bay vs Minnesota 2018
games.loc[(games['year'] == 2018) & (games['home_team_name'] == 'Vikings') & (games['away_team_name'] == 'Packers')]


,year,week,home_team,away_team,winner,tie,day,date,time,pts_win,pts_loss,yds_win,turnovers_win,yds_loss,turnovers_loss,home_team_name,home_team_city,away_team_name,away_team_city
4964,2018,12,Minnesota Vikings,Green Bay Packers,Minnesota Vikings,1,Sun,November 25,8:20PM,24,17,416,0,254,1,Vikings,Minnesota,Packers,Green Bay


In [26]:
standings.info()
standings.isna().sum()

# Output looks good. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638 entries, 0 to 637
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   team                  638 non-null    object 
 1   team_name             638 non-null    object 
 2   year                  638 non-null    int64  
 3   wins                  638 non-null    int64  
 4   loss                  638 non-null    int64  
 5   points_for            638 non-null    int64  
 6   points_against        638 non-null    int64  
 7   points_differential   638 non-null    int64  
 8   margin_of_victory     638 non-null    float64
 9   strength_of_schedule  638 non-null    float64
 10  simple_rating         638 non-null    float64
 11  offensive_ranking     638 non-null    float64
 12  defensive_ranking     638 non-null    float64
 13  playoffs              638 non-null    object 
 14  sb_winner             638 non-null    object 
dtypes: float64(5), int64(6)

team                    0
team_name               0
year                    0
wins                    0
loss                    0
points_for              0
points_against          0
points_differential     0
margin_of_victory       0
strength_of_schedule    0
simple_rating           0
offensive_ranking       0
defensive_ranking       0
playoffs                0
sb_winner               0
dtype: int64

In [80]:
# I am trying to see what the time horizons are for each of the datasets
    # All of them are the same which is good

standings['year'].min() #2000
standings['year'].max() #2019

games['year'].min() #2000
games['year'].max() #2019

attendance['year'].min() #2000
attendance['year'].max() #2019

2019

## Data Wrangling

Adding data from the record table to the attendace table and saving as a new copy

In [67]:
# Need to make a primary key to join on for both tables

# Attendance Table
attendance['TeamYear'] = attendance['team']+attendance['year'].astype(str)

# Standings Table
standings['TeamYear'] = standings['team']+standings['year'].astype(str)

In [69]:
attend_rec = pd.merge(attendance, standings, on='TeamYear', how='left')
attend_rec

,team_x,team_name_x,year_x,total,home,away,week,weekly_attendance,TeamYear,team_y,...,points_for,points_against,points_differential,margin_of_victory,strength_of_schedule,simple_rating,offensive_ranking,defensive_ranking,playoffs,sb_winner
0,Arizona,Cardinals,2000,893926,387475,506451,1,77434.0,Arizona2000,Arizona,...,210,443,-233,-14.6,-0.7,-15.2,-7.2,-8.1,No Playoffs,No Superbowl
1,Arizona,Cardinals,2000,893926,387475,506451,2,66009.0,Arizona2000,Arizona,...,210,443,-233,-14.6,-0.7,-15.2,-7.2,-8.1,No Playoffs,No Superbowl
2,Arizona,Cardinals,2000,893926,387475,506451,3,NaN,Arizona2000,Arizona,...,210,443,-233,-14.6,-0.7,-15.2,-7.2,-8.1,No Playoffs,No Superbowl
3,Arizona,Cardinals,2000,893926,387475,506451,4,71801.0,Arizona2000,Arizona,...,210,443,-233,-14.6,-0.7,-15.2,-7.2,-8.1,No Playoffs,No Superbowl
4,Arizona,Cardinals,2000,893926,387475,506451,5,66985.0,Arizona2000,Arizona,...,210,443,-233,-14.6,-0.7,-15.2,-7.2,-8.1,No Playoffs,No Superbowl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11623,Washington,Redskins,2019,1101466,523906,577560,13,71504.0,Washington2019,Washington,...,266,435,-169,-10.6,-0.2,-10.8,-6.3,-4.5,No Playoffs,No Superbowl
11624,Washington,Redskins,2019,1101466,523906,577560,14,77296.0,Washington2019,Washington,...,266,435,-169,-10.6,-0.2,-10.8,-6.3,-4.5,No Playoffs,No Superbowl
11625,Washington,Redskins,2019,1101466,523906,577560,15,63246.0,Washington2019,Washington,...,266,435,-169,-10.6,-0.2,-10.8,-6.3,-4.5,No Playoffs,No Superbowl
11626,Washington,Redskins,2019,1101466,523906,577560,16,66083.0,Washington2019,Washington,...,266,435,-169,-10.6,-0.2,-10.8,-6.3,-4.5,No Playoffs,No Superbowl


I would like to incorporate the yards won and lost in a given season and sum those for every team. 